In [5]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub

# 设置镜像源（清华源）
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
os.environ['TFHUB_CACHE_DIR'] = '/tmp/tfhub'

# 使用国内镜像地址
model_url = "https://hub.tensorflow.google.cn/google/elmo/3"
elmo = hub.load(model_url).signatures["default"]

def get_elmo_embedding(text):
    # 输入必须是 list of strings
    result = elmo(tf.constant([text]))
    # 打印 keys 来确认
    print("ELMo output keys:", result.keys())
    # 使用 'elmo' 键来获取最终向量表示
    embeddings = result["elmo"]
    return tf.reduce_mean(embeddings, axis=1).numpy()[0]

def evaluate_sentence_pair(s1, s2):
    emb1 = get_elmo_embedding(s1)
    emb2 = get_elmo_embedding(s2)
    
    score1 = np.linalg.norm(emb1)
    score2 = np.linalg.norm(emb2)
    
    return 0 if score1 > score2 else 1

# 测试
s1 = "The cat sat on the mat."
s2 = "The mat sat on the cat."
pred = evaluate_sentence_pair(s1, s2)
print("Valid sentence is:", "s1" if pred == 0 else "s2")


ELMo output keys: dict_keys(['lstm_outputs1', 'lstm_outputs2', 'word_emb', 'elmo', 'sequence_len', 'default'])
ELMo output keys: dict_keys(['lstm_outputs1', 'lstm_outputs2', 'word_emb', 'elmo', 'sequence_len', 'default'])
Valid sentence is: s2


In [2]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os

# 设置镜像源
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
os.environ['TFHUB_CACHE_DIR'] = '/tmp/tfhub'

# 使用国内镜像地址加载 ELMo 模型
model_url = "https://hub.tensorflow.google.cn/google/elmo/3"
elmo = hub.load(model_url).signatures["default"]

# 获取句向量（平均池化）
import re

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r"[^a-zA-Z0-9\s.,!?']", '', text)  # 只保留基础英文字符
    return text

def get_elmo_embedding(text):
    cleaned = clean_text(text)
    if cleaned == "":
        return np.zeros(1024)  # ELMo 输出维度是 1024
    try:
        embeddings = elmo(tf.constant([cleaned]))["elmo"]
        return tf.reduce_mean(embeddings, axis=1).numpy()[0]
    except Exception as e:
        print(f"ELMo embedding error for text: '{text}'\n{e}")
        return np.zeros(1024)


# 判断哪句更合理（返回 0 表示 sentence0 更合理，1 表示 sentence1 更合理）
def evaluate_sentence_pair(s1, s2):
    emb1 = get_elmo_embedding(s1)
    emb2 = get_elmo_embedding(s2)
    score1 = np.linalg.norm(emb1)
    score2 = np.linalg.norm(emb2)
    return 0 if score1 > score2 else 1

# 给定错误句子和解释选项，选择与其最相似的解释（返回 'A'/'B'/'C'）
def select_reason(false_sentence, options):
    sentence_emb = get_elmo_embedding(false_sentence)
    scores = {}
    for key in ['A', 'B', 'C']:
        reason_emb = get_elmo_embedding(options[key])
        similarity = np.dot(sentence_emb, reason_emb) / (np.linalg.norm(sentence_emb) * np.linalg.norm(reason_emb))
        scores[key] = similarity
    return max(scores, key=scores.get)

# 读取数据文件
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

# 加载数据（注意替换文件名）
data = load_jsonl('dataset.jsonl')

# # 评估句子选择准确率
# correct_sentence = 0
# for item in data:
#     pred = evaluate_sentence_pair(item["sentence0"], item["sentence1"])
#     if pred != item["false"]:
#         correct_sentence += 1
# sentence_accuracy = correct_sentence / len(data)
# print(f"ELMo model sentence selection accuracy: {sentence_accuracy:.2f}")

# 评估解释选择准确率
correct_reason = 0
for item in data:
    false_sentence = item["sentence0"] if item["false"] == 0 else item["sentence1"]
    pred_reason = select_reason(false_sentence, {"A": item["A"], "B": item["B"], "C": item["C"]})
    if pred_reason == item["reason"]:
        correct_reason += 1
reason_accuracy = correct_reason / len(data)
print(f"ELMo model explanation selection accuracy: {reason_accuracy:.2f}")


C:\Users\ck\AppData\Local\Temp\ipykernel_22412\3301223317.py:51: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(sentence_emb, reason_emb) / (np.linalg.norm(sentence_emb) * np.linalg.norm(reason_emb))


ELMo model explanation selection accuracy: 0.36


In [3]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
import re

# 配置 ELMo 模型加载
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
os.environ['TFHUB_CACHE_DIR'] = '/tmp/tfhub'
model_url = "https://hub.tensorflow.google.cn/google/elmo/3"
elmo = hub.load(model_url).signatures["default"]

# 清洗文本
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r"[^a-zA-Z0-9\s.,!?']", '', text)
    return text

# 获取ELMo向量（平均池化）
def get_elmo_embedding(text):
    text = clean_text(text)
    if text == "":
        return np.zeros(1024)
    try:
        embeddings = elmo(tf.constant([text]))["elmo"]
        return tf.reduce_mean(embeddings, axis=1).numpy()[0]
    except Exception as e:
        print(f"Embedding error for: {text}\n{e}")
        return np.zeros(1024)

# 加载 jsonl 数据
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

# 构造三条句子 + label
def build_reason_sentences(data):
    all_sentences = []
    all_labels = []
    for item in data:
        false_sent = item["sentence0"] if item["false"] == 0 else item["sentence1"]
        reasons = [
            f"{false_sent} is against common sense because {item['A']}.",
            f"{false_sent} is against common sense because {item['B']}.",
            f"{false_sent} is against common sense because {item['C']}."
        ]
        label = ord(item["reason"]) - ord("A")  # A=0, B=1, C=2
        all_sentences.append(reasons)
        all_labels.append(label)
    return all_sentences, all_labels

# 模型推理
def evaluate_reason_selection(all_sentences, all_labels):
    correct = 0
    for reasons, label in zip(all_sentences, all_labels):
        sims = []
        for reason in reasons:
            emb = get_elmo_embedding(reason)
            sim = np.linalg.norm(emb)
            sims.append(sim)
        pred = np.argmax(sims)
        if pred == label:
            correct += 1
    return correct / len(all_labels)

# 主程序
data = load_jsonl("dataset.jsonl")
reason_sentences, reason_labels = build_reason_sentences(data)
accuracy = evaluate_reason_selection(reason_sentences, reason_labels)
print(f"ELMo model explanation sel+ection (concatenated form) accuracy: {accuracy:.2f}")


ELMo model explanation selection (concatenated form) accuracy: 0.40


In [8]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


# 设置镜像源（清华源）
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
os.environ['TFHUB_CACHE_DIR'] = '/tmp/tfhub'

# 使用国内镜像地址
model_url = "https://hub.tensorflow.google.cn/google/elmo/3"
elmo = hub.load(model_url).signatures["default"]


# 获取句向量（平均池化）
def get_elmo_embedding(text):
    embeddings = elmo(tf.constant([text]))["elmo"]
    return tf.reduce_mean(embeddings, axis=1).numpy()[0]

# 判断哪句更合理（返回 0 表示 sentence0 更合理，1 表示 sentence1 更合理）
def evaluate_sentence_pair(s1, s2):
    emb1 = get_elmo_embedding(s1)
    emb2 = get_elmo_embedding(s2)
    score1 = np.linalg.norm(emb1)
    score2 = np.linalg.norm(emb2)
    return 0 if score1 > score2 else 1

def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]
# 数据列表（可以从文件中读取）
data = load_jsonl('dataset.jsonl')

# 评估准确率
correct = 0
for item in data:
    pred = evaluate_sentence_pair(item["sentence0"], item["sentence1"])
    # 如果模型选择的句子不是错误的那一句，则为正确
    if pred != item["false"]:
        correct += 1

accuracy = correct / len(data)
print(f"ELMo model accuracy on your dataset: {accuracy:.2f}")


ELMo model accuracy on your dataset: 0.62


In [3]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity


# 获取句向量（平均池化）
def get_elmo_embedding(text):
    embeddings = elmo(tf.constant([text]))["elmo"]  # shape: [1, timesteps, 1024]
    return tf.reduce_mean(embeddings, axis=1).numpy()[0]  # shape: [1024]

# 使用句向量与零向量之间的余弦相似度作为“自然程度”判断
def evaluate_sentence_pair(s1, s2):
    emb1 = get_elmo_embedding(s1)
    emb2 = get_elmo_embedding(s2)
    # 比较哪个向量更“接近”中心语言空间方向（原点除外的方向）
    sim1 = cosine_similarity([emb1], [[0]*1024])[0][0]
    sim2 = cosine_similarity([emb2], [[0]*1024])[0][0]
    return 0 if sim1 > sim2 else 1

# 加载数据集
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

data = load_jsonl('dataset.jsonl')

# 评估准确率
correct = 0
for item in data:
    pred = evaluate_sentence_pair(item["sentence0"], item["sentence1"])
    if pred != item["false"]:
        correct += 1

accuracy = correct / len(data)
print(f"ELMo + cosine similarity accuracy: {accuracy:.2f}")


ELMo + cosine similarity accuracy: 0.50


In [6]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity



# 获取 forward 和 backward 的平均向量
def get_forward_backward_embeddings(text):
    elmo_output = elmo(tf.constant([text]))
    fw = elmo_output["lstm_outputs1"][:, :, :512]  # forward LSTM hidden state
    bw = elmo_output["lstm_outputs1"][:, :, 512:]  # backward LSTM hidden state
    # 平均池化
    fw_avg = tf.reduce_mean(fw, axis=1).numpy()[0]
    bw_avg = tf.reduce_mean(bw, axis=1).numpy()[0]
    return fw_avg, bw_avg

# 判断哪句更合理（forward 与 backward 的余弦夹角更小者更合理）
def evaluate_sentence_pair(s1, s2):
    fw1, bw1 = get_forward_backward_embeddings(s1)
    fw2, bw2 = get_forward_backward_embeddings(s2)
    sim1 = cosine_similarity([fw1], [bw1])[0][0]
    sim2 = cosine_similarity([fw2], [bw2])[0][0]
    return 0 if sim1 > sim2 else 1  # 相似度越高，forward/backward 越一致

# 加载数据
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

# 加载数据并评估
data = load_jsonl('dataset.jsonl')

correct = 0
for item in data:
    pred = evaluate_sentence_pair(item["sentence0"], item["sentence1"])
    if pred != item["false"]:
        correct += 1

accuracy = correct / len(data)
print(f"ELMo forward/backward consistency accuracy: {accuracy:.2f}")


ELMo forward/backward consistency accuracy: 0.58


In [7]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub



# 获取 forward 和 backward 的平均向量
def get_forward_backward_embeddings(text):
    elmo_output = elmo(tf.constant([text]))
    fw = elmo_output["lstm_outputs1"][:, :, :512]  # forward LSTM hidden state
    bw = elmo_output["lstm_outputs1"][:, :, 512:]  # backward LSTM hidden state
    fw_avg = tf.reduce_mean(fw, axis=1).numpy()[0]
    bw_avg = tf.reduce_mean(bw, axis=1).numpy()[0]
    return fw_avg, bw_avg

# 判断哪句更合理（forward 和 backward 的欧几里得距离更小）
def evaluate_sentence_pair(s1, s2):
    fw1, bw1 = get_forward_backward_embeddings(s1)
    fw2, bw2 = get_forward_backward_embeddings(s2)
    dist1 = np.linalg.norm(fw1 - bw1)
    dist2 = np.linalg.norm(fw2 - bw2)
    return 0 if dist1 < dist2 else 1  # 距离越小越好

# 加载数据
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

# 加载数据并评估
data = load_jsonl('dataset.jsonl')

correct = 0
for item in data:
    pred = evaluate_sentence_pair(item["sentence0"], item["sentence1"])
    if pred != item["false"]:
        correct += 1

accuracy = correct / len(data)
print(f"ELMo forward/backward Euclidean distance accuracy: {accuracy:.2f}")


ELMo forward/backward Euclidean distance accuracy: 0.45


In [ ]:
import spacy
import requests

nlp = spacy.load("en_core_web_sm")

def get_elmo_embedding(text):
    embeddings = elmo(tf.constant([text]))["elmo"]
    return tf.reduce_mean(embeddings, axis=1).numpy()[0]

# 从 ConceptNet 获取所有关系
def get_all_conceptnet_relations(entities):
    relations = []
    if len(entities) < 2:
        return relations
    for i in range(len(entities)):
        for j in range(i + 1, len(entities)):
            start_node = entities[i].lower().replace(" ", "_")
            end_node = entities[j].lower().replace(" ", "_")
            url = f"http://api.conceptnet.io/query?node1=/c/en/{start_node}&node2=/c/en/{end_node}&rel=/r/RelatedTo"
            try:
                response = requests.get(url, timeout=5)
                data = response.json()
                if data["edges"]:
                    for edge in data["edges"]:
                        relation = edge["rel"]["label"]
                        relations.append(f"{end_node} is related to {start_node} via {relation}.")
            except Exception as e:
                print(f"ConceptNet error for {start_node} and {end_node}: {e}")
                relations.append(f"{end_node} is related to {start_node} via related to.")
    return relations

# 实体识别并编码所有知识
def encode_with_all_knowledge(sentence):
    doc = nlp(sentence)
    entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "PRODUCT", "FAC", "EVENT", "WORK_OF_ART"]]
    if not entities:
        return enhanced_sentence
    knowledge_sentences = get_all_conceptnet_relations(entities)
    if not knowledge_sentences:
        return sentence

    enhanced_sentence = sentence
    for knowledge in knowledge_sentences:
        enhanced_sentence += f" {knowledge}"
    return enhanced_sentence

# 加载 jsonl 数据
def load_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

# 构建理由句子并添加知识编码
def build_reason_sentences(data):
    all_sentences = []
    all_labels = []
    for item in data:
        false_sent = item["sentence0"] if item["false"] == 0 else item["sentence1"]
        # 编码所有知识
        false_sent_encoded = encode_with_all_knowledge(false_sent)
        reasons = [
            f"{false_sent_encoded} is against common sense because {item['A']}.",
            f"{false_sent_encoded} is against common sense because {item['B']}.",
            f"{false_sent_encoded} is against common sense because {item['C']}."
        ]
        label = ord(item["reason"]) - ord("A")  # A=0, B=1, C=2
        all_sentences.append(reasons)
        all_labels.append(label)
    return all_sentences, all_labels

# 评估理由选择（基于嵌入范数）
def evaluate_reason_selection(all_sentences, all_labels):
    correct = 0
    for reasons, label in zip(all_sentences, all_labels):
        sims = []
        for reason in reasons:
            emb = get_elmo_embedding(reason)
            # 使用嵌入的范数作为得分
            sim = np.linalg.norm(emb)
            sims.append(sim)
        pred = np.argmax(sims)
        if pred == label:
            correct += 1
    return correct / len(all_labels)

In [ ]:
# 加载数据
data = load_jsonl('dataset.jsonl')

# 构建知识编码后的句子
reason_sentences, reason_labels = build_reason_sentences(data)

# 评估
accuracy = evaluate_reason_selection(reason_sentences, reason_labels)
print(f"ELMo model (with all knowledge enhancement) explanation selection accuracy: {accuracy:.4f}")

In [ ]:
def evaluate_sentence_pair(s1, s2):
    # 增强知识
    s1_encoded = encode_with_all_knowledge(s1)
    s2_encoded = encode_with_all_knowledge(s2)
    # 获取嵌入
    emb1 = get_elmo_embedding(s1_encoded)
    emb2 = get_elmo_embedding(s2_encoded)
    # 使用嵌入范数作为得分
    score1 = np.linalg.norm(emb1)
    score2 = np.linalg.norm(emb2)
    return 0 if score1 > score2 else 1  # 得分越高越合理，返回 0 表示 sentence0 更合理

correct_sentence = 0
for item in data:
    pred = evaluate_sentence_pair(item["sentence0"], item["sentence1"])
    # 如果模型选择的句子不是错误的那一句，则为正确
    if pred != item["false"]:
        correct_sentence += 1
sentence_accuracy = correct_sentence / len(data)
print(f"ELMo model (with knowledge enhancement) sentence selection accuracy: {sentence_accuracy:.4f}")